# Identifying Strongly-Lensed Graviational-Wave Signals using `hanabi`

`hanabi`, which stands for **H**ierarchical bayesian **ANA**lysis on lensed GW signals using **BI**lby, implements a hierarchical Bayesian approach that accounts for both selection effects and population information to identify strongly-lensed GW signals using the bilby codebase. Here we will briefly explain the statistical framework. For more details on the methodology, see *A Bayesian Statistical Framework for Identifying Strongly-Lensed Gravitational-Wave Signals* by Lo et al. (to be published/posted to arXiv).

The basic idea is to compute a statistic, commonly referred as a *Bayes factor*, that allows us to choose one out of the two competing hypotheses that claim to describe the data that we observed. In our case, where we try to discern strongly-lensed gravitational-wave signals, the two hypotheses would be the lensed hypothesis $\mathcal{H}_{\rm L}$ and the not-lensed hypothesis $\mathcal{H}_{\rm NL}$. In particular, if there are two GW events under consideration, then the two hypotheses mean that

$$
\begin{aligned}
\mathcal{H}_{\rm L} & : & \textrm{The two GW signals are lensed images coming from the same source} \\
\mathcal{H}_{\rm NL} & : & \textrm{The two GW signals came from two separate sources}
\end{aligned}
,
$$
where we see that whether or not two signals are lensed would depend on the particular source population (model) one assumes.

The Bayes factor $\mathcal{B}$ is given by the ratio of the probability (density) of observing the data set of interest $\mathbf{D}$ under the two hypotheses, namely

$$
\mathcal{B} = \frac{p(\mathbf{D}|\mathcal{H}_{\rm L})}{p(\mathbf{D}|\mathcal{H}_{\rm NL})}.
$$

Note that the probability densities $p(\mathbf{D}|\mathcal{H}_{\rm L}), p(\mathbf{D}|\mathcal{H}_{\rm NL})$, which are also known as the **evidence**, should be normalized (proper). This is where selection effect comes in -- the selection functions serve as the normalization constants for each of the hypothesis. In order to compute the probability densities of observing the data $\mathbf{D}$, one will need to have a population model for the source, and a lensing model. Optionally, instead of fixing one particular source population model and lensing model, it is straightforward to consider a family of source population models and lensing models parametrized by some 'hyper-parameters' and marginalize over the hyper-parameters.


In the following notebook, we demonstrate how to use the `hanabi` package to compute the (proper) Bayes factor. Optionally, one can multiply the Bayes factor with an additional "timing Bayes factor" to leverage our prior knowledge about the time delay of lensed signals, but we will not be including the calculation in this notebook.

In [ ]:
# Loading libraries
import numpy as np
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format='retina'
from matplotlib import pyplot as plt
import pandas as pd
import h5py

import bilby
from hanabi.hierarchical.source_population_model import *
from hanabi.hierarchical.merger_rate_density import *
from hanabi.hierarchical.selection_function import *
from hanabi.hierarchical.p_z import *
from hanabi.hierarchical.result import *
from hanabi.lensing.optical_depth import *
from hanabi.lensing.absolute_magnification import *
from hanabi.hierarchical.marginalized_likelihood import *
from hanabi.hierarchical.gibbs_sampling import CustomCollapsedBlockedGibbsSampler
from hanabi.hierarchical.reweight_with_population_model import *

In [ ]:
defaults_kwargs = {
    'bins': 50,
    'smooth': 0.9,
    'label_kwargs': {'fontsize': 16},
    'title_kwargs': {'fontsize': 16},
    'color': '#0072C1',
    'truth_color': 'tab:orange',
    'quantiles': [0.16, 0.84],
    'levels': (1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.)),
    'plot_density': False,
    'plot_datapoints': True,
    'fill_contours': True,
    'max_n_ticks': 3
}

## Loading PE results

First, we read in the joint-PE result, which was generated using `hanabi.inference`, and the single-PE results, which were generated using `bilby_pipe`. After a PE run has been completed, both will generate a JSON file containing the inference result such as the posterior samples and the evidence.


The `hanabi.hierarchical` module **requires that the component-mass parametrization is used**. If a different mass parametrization (oftentimes chirp_mass-mass_ratio) was used during the sampling instead, you can run
```bash
$ hanabi_postprocess_result --flat-in-component-masses PATH_TO_RESULT_JSON_FILE
```
to convert a `bilby` result using the chirp_mass-mass_ratio parametrization to the component-mass parametrization, taking the Jacobian into account. Note that the prior for the component masses does not have to be flat/uniform.

The module also **requires the apparent luminosity distance of each signal/image is being sampled**. The apparent luminosity distance of the `i`-th image (`i=1,2,...`) should be called `luminosity_distance^(i)` as its name.

In [ ]:
# Loading PE results
rundir = "RUNDIR"
event_1 = "trigger_1"
event_2 = "trigger_2"

single_event_PE_result_path_template = "{rundir}/highmass-lensed-{event}_0_reweighted_result.json"
joint_PE_result_path_template = "{rundir}/joint_{event_1}_{event_2}_0_reweighted_result.json"

result_1 = bilby.result.read_in_result(single_event_PE_result_path_template.format(rundir=rundir, event=event_1))
result_2 = bilby.result.read_in_result(single_event_PE_result_path_template.format(rundir=rundir, event=event_2))
joint_result = bilby.result.read_in_result(joint_PE_result_path_template.format(rundir=rundir, event_1=event_1, event_2=event_2))

### Computing the coherence ratio

One can compute the so-called 'coherence ratio' as an intermediate statistic, which is the *unnormalized/improper* Bayes factor under a particular set of sampling priors **without** selection effects and population information. It cannot/should not be interpreted as a ratio of probability like a proper Bayes factor. For example, a coherence ratio of 10 does not mean that the lensed hypothesis is ten times more favored by the observed data than the not-lensed hypothesis. However, **a negative log coherence ratio would indicate that the observed data set cannot be explained by a common set of intrinsic parameters** (and some of the extrinsic parameters such as the sky location) as in the case if the events are truly lensed. That is, the events are 'incoherent' and hence the name of the statistic.

Here we calculate the log coherence ratio from the PE results and print out its value.

In [ ]:
lcr = compute_log_coherence_ratio(joint_result, result_1, result_2)
print(lcr)

## Setting up the population and lensing model
In order to compute the evidence under the lensed and not-lensed hypothesis, one will need to provide a *source population model*, which **describes the fraction of sources satisfying some properties** (hence normalized), and a *lensing model*, which **describes the fraction of sources that is lensed with images satisfying some properties** (again normalized).

<font color='red'><b>[Change the following before public release to reflect what was being used]</b></font>

Here we use the following population model for masses

$$
p(m_1^{\rm src}, m_2^{\rm src}|\alpha, \beta, m_{\rm min}, m_{\rm max}) = 
\begin{cases}
\frac{1 - \alpha}{m_{\rm max}^{1-\alpha} - m_{\rm min}^{1-\alpha}} m_1^{-\alpha} \frac{1 + \beta}{(m_{1}^{\rm src})^{1+\beta} - m_{\rm min}^{1+\beta}} m_2^{\beta} & \textrm{if } m_{\rm min} \leq m_2^{\rm src} \leq m_1^{\rm src} \leq m_{\rm max} \\
0 & \textrm{otherwise}
\end{cases},
$$

with $\alpha=1.8, \beta=0, m_{\rm min}=5 M_{\odot}, m_{\rm max}=60 M_{\odot} $.

For spin, we are using the isotropic spin distribution, which is the same as the sampling distribution for the spins.

For the merger rate density $\mathcal{R}(z)$, which in turns controls how the source redshift $z$ is distributed. Here we use a merger rate density model tracking the star-formation rate of *Belczynski et al. 2017* for population-I and population-II stars, namely

$$
\mathcal{R}(z) = \frac{6.6\times10^{3}\exp(1.6z)}{30+\exp(2.1z)}.
$$

Note that here we use the pre-defined models in `hanabi.hierarchical.source_population_model` but users are free to use any compatible population model (see [here](https://git.ligo.org/ka-lok.lo/hanabi/-/blob/master/hanabi/hierarchical/source_population_model.py) for more information).


As for the lensing model, we assume that the absolute magnification of an image $\mu$ follows the following probability distribution

$$
p(\mu) \propto 
\begin{cases}
\mu^{-3} & \textrm{if }\mu \geq 2 \\
0 & \textrm{otherwise}
\end{cases},
$$

pre-defined in `hanabi.lensing.absolute_magnification` and adopt an optical depth $\tau(z)$ (the probability of strong lensing for a source at redshift $z$) from *Hannuksela et al. 2019*, namely

$$
\tau(z) = F \left( \frac{d_{\rm C}(z)}{d_{\rm H}} \right)^{3},
$$

where $F$ is an empirical constant taken to be $0.0017$ here, $d_{\rm H}$ is the Hubble distance and $d_{\rm C}(z)$ is the comoving distance. This model is pre-defined in `hanabi.lensing.optical_depth`.

Again, users are free to use any compatible model. For the absolute magnification, users can provide any `bilby` prior. For the optical depth, users can provide any class that implements a function `evaluate(z)` that returns the optical depth at $z$ (see [here](https://git.ligo.org/ka-lok.lo/hanabi/-/blob/master/hanabi/lensing/optical_depth.py) for more information).

In [ ]:
# Setting up the population models
mass_src_pop_model = PowerLawComponentMass(alpha=1.8, beta=0, mmin=5, mmax=60)
spin_src_pop_model = None # Spin information is not being used
merger_rate_density = BelczynskiEtAl2017PopIPopIIStarsBBHMergerRateDensity()

# Setting up the strong lensing models
optical_depth = HannukselaEtAl2019OpticalDepth()
pz_lensed = LensedSourceRedshiftProbDist(merger_rate_density=merger_rate_density, optical_depth=optical_depth)
pz_notlensed = NotLensedSourceRedshiftProbDist(merger_rate_density=merger_rate_density, optical_depth=optical_depth)
abs_magnifications = [SISPowerLawAbsoluteMagnification(), SISPowerLawAbsoluteMagnification()]

## Computing the (unnormalized) evidence under the not-lensed hypothesis

We now compute the (unnormalized) evidence under the not-lensed hypothesis from the single-event PE results. Note that the aforementioned degeneracy does not exist under the not-lensed hypothesis. Therefore, the calculation of the evidence will not involve any extra marginalization over redshift but a simple reweighting with the given source population model, as we can compute $z = z(d_{\rm L})$ as a function of the luminosity distance $d_{\rm L}$. This reweighting is implemented in `hanabi.hierarchical.reweight_with_population_model`.

In [ ]:
# Reweighting single-event results with population models
reweighting_result_1 = ReweightWithPopulationModel(result_1, mass_src_pop_model, spin_src_pop_model, pz_notlensed)
reweighting_result_2 = ReweightWithPopulationModel(result_2, mass_src_pop_model, spin_src_pop_model, pz_notlensed)

## Computing the (unnormalized) evidence under the lensed hypothesis

We first compute the (unnormalized) evidence under the lensed hypothesis from a joint-PE result.

Note that during the sampling/inference, detector-frame quantities were inferred, such as the *redshiftted* masses and the *apparent* luminosity distance (which is different from the true source luminosity distance due to magnification by lensing). In general we will not be able to 'de-magnified' and infer the true source luminosity distance, and hence the true source redshift and masses due to this magnification effect.

Even though we might not be able to pinpoint the exact redshift of the source, notice that given a source population model and a source redshift, one can easily work out the appropriate prior for the detector-frame quantities which are being sampled. Effectively, we **treat the source redshift $z$ as a hyper-parameter for the prior**. This is akin to the normal population analysis for GWs.

Therefore our hierarchical analysis consists of only one parameter, the true source redshift $z$. The 'hyper-prior' $p_{z}(z)$ under the lensed hypothesis can be calculated from the given $\mathcal{R}(z)$ and $\tau(z)$, whereas the likelihood $\mathcal{L}(z)$ can be constructed from the joint-PE result. In particular, we can think of the evidence from PE as a function of the hyper-parameter $z$ as we changes the prior for the detector-frame quantities with $z$. Since we used a Monte-Carlo algorithm (nested sampling to be exact) for PE, this likelihood $\mathcal{L}(z)$ is aptly named Monte-Carlo marginalized likelihood.

In the following, we use `bilby` with a nested sampler called `dynesty` to do this 'marginalization' over the source redshift to get the unnormalized evidence under the lensed hypothesis, which is the prime product of a nested sampling algorithm. We will correct for the normalization later on. As a 'by-product' of the nested sampling, we also get a set of posterior samples of the redshift $z \sim p(z|\mathbf{D})$. Later we will show how one can use Gibbs sampling to get back the desired unbiased source-frame parameters.

In [ ]:
ncores = 8
label = "marginalize_over_redshift_{}_{}".format(event_1, event_2)
# Forcing multiprocessing to use the spawn method to avoid signature clash
import multiprocessing
multiprocessing.set_start_method("spawn")

In [ ]:
# Setting up likelihood
likelihood = MonteCarloMarginalizedLikelihood(joint_result, mass_src_pop_model, spin_src_pop_model, abs_magnifications)
# Marginalizing over redshift for the joint result
redshift_result = bilby.run_sampler(likelihood=likelihood, priors={'redshift': pz_lensed}, sampler='dynesty', nlive=1000, nact=30, npool=ncores, sample='unif', outdir="marginalization", label=label)

Now, we can repeat the log coherence ratio calculation, but this time with priors accounting for the population information.

In [ ]:
log_coherence_ratio = compute_log_coherence_ratio(redshift_result, reweighting_result_1, reweighting_result_2)
print(log_coherence_ratio)

## Computing the (proper) Bayes factor

With the unnormalized Bayes factor calculated above, we will need to compute the appropriate normalization constants (which are called selection functions) under each of the hypotheses. Under the not-lensed hypothesis, the two GW signals under consideration came from two separate sources. The selection function under the not-lensed hypothesis for a single source is referred as $\alpha$. Since there are two sources, the proper normalization constant would be $1/\alpha^2$. Similarly for the lensed hypothesis, the selection function is termed $\beta$ and hence the proper normalization constant would be $1/\beta$.

In order to compute the properly normalized Bayes factor, one will need to multiple the value of the coherence ratio by the normalization constant $\alpha^2/\beta$.

The value $\ln \beta/\alpha^2$ provides us a scale for the log coherence ratio. This is because if the (population-weighted) log coherence ratio is smaller than this value, then the actual proper log Bayes factor would be negative

Here we load a HDF5 file with selection functions $\alpha, \beta$ pre-computed using the package `pdetclassifier`. We provide an interface with `pdetclassifier` called `compute_selection_functions.py` under the folder `hanabi/hierarchical/pdetclassifier`.

We can now calculate the proper (normalized) Bayes factor for the lensed versus the not-lensed hypothesis, which is the coherence ratio corrected for normalization as

In [ ]:
log_Bayes_factor = compute_log_Bayes_factor(
                    "/home/ka-lok.lo/projects/stronglensing/hanabi/hanabi/hierarchical/pdetclassifier/o3a/o3a_first3months_bbhpop_powerlaw_mmax_60_isotropic_spin_avg.h5",
                    redshift_result,
                    reweighting_result_1,
                    reweighting_result_2)

print(log_Bayes_factor)

## Inferring unbiased source parameters using Gibbs sampling

Ultimately, we are interested in the true/unbiased source-frame parameters such as the true component masses and not the redshiftted masses. However, we only have a set of posterior samples for the source redshift $z \sim p(z|\mathbf{D})$, and a set of detector-frame parameters from the PE $\theta \sim p(\theta|\mathbf{D}, z)$. With the help of Gibbs sampling, we can draw the desired joint posterior distribution

$$
z, \theta \sim p(z, \theta|\mathbf{D}) \propto p(\theta|\mathbf{D},z)p(z|\mathbf{D})
$$

From the joint posterior samples, we can easily compute the unbiased source-frame parameters.

Here we call the custom Gibbs sampler implemented in `hanabi.hierarchical.gibbs_sampling` and draw 10000 joint posterior samples

In [ ]:
# Gibbs sampling
sampler = CustomCollapsedBlockedGibbsSampler(redshift_result, likelihood, pool=ncores)
joint_samples = sampler.sample(10000)

In [ ]:
compute_source_parameters(joint_samples)

In [ ]:
import corner
plt.rcParams.update({'text.usetex': True})
plt.figure(dpi=150)
fig = corner.corner(joint_samples[["mass_1_source", "mass_2_source", "total_mass_source", "magnification^(1)", "magnification^(2)", "relative_magnification^(2)", "redshift"]], labels=[r'$m_1^{\rm src}\;[M_{\odot}]$', r'$m_2^{\rm src}\;[M_{\odot}]$', r'$M_{\rm tol}^{\rm src}\;[M_{\odot}]$', r'$\mu^{(1)}$', r'$\mu^{(2)}$', r'$\mu^{(\rm rel)}$', r'$z$'], **defaults_kwargs)

In [ ]:
save_hierarchical_analysis_result(
    label="{}_{}_hierarchical_analysis".format(event_1, event_2),
    log_Bayes_factor=log_Bayes_factor,
    log_coherence_ratio=log_coherence_ratio,
    joint_samples=joint_samples
)